In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## init stereo aruco detector

In [3]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
# stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [6]:
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (1.5,1.5,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
# wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.2,-0.5,0), 
#                           rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
# wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (-0.2,-0.5,0), 
#                           rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (-0.2,-0.5,0), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [7]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.2,0.07,0.02), 
                   center=(0,-0.5,0.045), rpy=(0,0,np.pi/2), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.2,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
# pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
# sweep = pscene.create_object(oname="sweep", gname="floor", _type=SweepTask, 
#                              action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
#                                                    "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})

### planning pipeline

In [16]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.object_a_star import ObjectAstar
tplan = ObjectAstar(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

binding_state
(('brush', 'face', 'floor', 'floor'),)
state_param
{'brush': ('base_link', array([[ 6.123234e-17, -1.000000e+00,  0.000000e+00,  0.000000e+00],
       [ 1.000000e+00,  6.123234e-17,  0.000000e+00, -5.000000e-01],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+00,  4.500000e-02],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]],
      dtype=float32))}
set_object_state
obj: brush
bd: ('brush', 'face', 'floor', 'floor')
state.Q: [ 0.          0.         -1.57079633  0.         -1.57079633  0.        ]


In [17]:
tplan.node_list

[('grip0',), ('goal',), ('floor',)]

In [18]:
tplan.node_dict

{('floor',): [('grip0',)],
 ('goal',): [('grip0',)],
 ('grip0',): [('floor',), ('goal',)]}

## ui

In [19]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Test plan

In [20]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.update_state(crob.home_pose)
goal_nodes = [("goal",)]
# goal_nodes = [("floor", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=True, dt_vis=0.01)
schedules = ppline.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

binding_state
(('brush', 'face', 'floor', 'floor'),)
state_param
{'brush': ('base_link', array([[ 6.123234e-17, -1.000000e+00,  0.000000e+00,  0.000000e+00],
       [ 1.000000e+00,  6.123234e-17,  0.000000e+00, -5.000000e-01],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+00,  4.500000e-02],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]],
      dtype=float32))}
from_node_item: floor
to_node_item: grip0
from_binding_state: ('brush', 'face', 'floor', 'floor')
get_available_bindings
oname: brush
node: grip0
bd_list: ['grip0']
Check:brush-grip0
binder:grip0
point:handle
leaf: ('grip0',)
state.binding_state: (('brush', 'face', 'floor', 'floor'),)
state.state_param: {'brush': ('base_link', array([[ 6.123234e-17, -1.000000e+00,  0.000000e+00,  0.000000e+00],
       [ 1.000000e+00,  6.123234e-17,  0.000000e+00, -5.000000e-01],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+00,  4.500000e-02],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]],
     

binding_state
(('brush', 'face', 'floor', 'floor'),)
state_param
{'brush': ('base_link', array([[ 6.123234e-17, -1.000000e+00,  0.000000e+00,  0.000000e+00],
       [ 1.000000e+00,  6.123234e-17,  0.000000e+00, -5.000000e-01],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+00,  4.500000e-02],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]],
      dtype=float32))}
node: ('floor',)->('grip0',) = fail
set_object_state
obj: brush
bd: ('brush', 'face', 'floor', 'floor')
state.Q: [ 0.          0.         -1.57079633  0.         -1.57079633  0.        ]
binding_state
(('brush', 'handle', 'grip0', 'grip0'),)
state_param
{'brush': ('indy0_tcp', array([[ 4.1388494e-01, -3.3468643e-04,  9.1032916e-01, -3.2256618e-02],
       [ 2.5549228e-04, -9.9999988e-01, -4.8381713e-04, -3.8937433e-04],
       [ 9.1032916e-01,  4.3281843e-04, -4.1388479e-01,  1.5419851e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32))}
from_node_item: g

binding_state
(('brush', 'handle', 'grip0', 'grip0'),)
state_param
{'brush': ('indy0_tcp', array([[ 4.1388494e-01, -3.3468643e-04,  9.1032916e-01, -3.2256618e-02],
       [ 2.5549228e-04, -9.9999988e-01, -4.8381713e-04, -3.8937433e-04],
       [ 9.1032916e-01,  4.3281843e-04, -4.1388479e-01,  1.5419851e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32))}
node: ('grip0',)->('goal',) = fail
set_object_state
obj: brush
bd: ('brush', 'handle', 'grip0', 'grip0')
state.Q: [0.13415982985975108, -1.6699426023460926, 0.44031498622236237, 0.9424388049413986, -2.991042478098566, 0.9931557963041637]
binding_state
(('brush', 'handle', 'grip0', 'grip0'),)
state_param
{'brush': ('indy0_tcp', array([[ 4.1388494e-01, -3.3468643e-04,  9.1032916e-01, -3.2256618e-02],
       [ 2.5549228e-04, -9.9999988e-01, -4.8381713e-04, -3.8937433e-04],
       [ 9.1032916e-01,  4.3281843e-04, -4.1388479e-01,  1.5419851e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.00

In [21]:
ppline.play_schedule(snode_schedule)

set_object_state
obj: brush
bd: ('brush', 'face', 'floor', 'floor')
state.Q: [ 0.          0.         -1.57079633  0.         -1.57079633  0.        ]
set_object_state
obj: brush
bd: ('brush', 'handle', 'grip0', 'grip0')
state.Q: [0.13415982985975108, -1.6699426023460926, 0.44031498622236237, 0.9424388049413986, -2.991042478098566, 0.9931557963041637]
set_object_state
obj: brush
bd: ('brush', 'face', 'goal_bd', 'goal')
state.Q: [2.1580753632574394, 0.7803396729476921, 1.6964210483458777, -1.7255760279822991, -1.1426650279216761, 0.7335089993408702]


In [22]:
print(snode_schedule[0].state.binding_state)
print(snode_schedule[1].state.binding_state)
print(snode_schedule[2].state.binding_state)

(('brush', 'face', 'floor', 'floor'),)
(('brush', 'handle', 'grip0', 'grip0'),)
(('brush', 'face', 'goal_bd', 'goal'),)


In [23]:
from pkg.planning.scene import State
idx = 0
from_state = snode_schedule[idx].state
to_state = snode_schedule[idx+1].state
redundancy_dict = snode_schedule[idx+1].redundancy_dict
redundancy_dict[obj]['w'] = 0
traj, lastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, 
                                                    redundancy_dict=redundancy_dict)
print(success)
if success:
    for bd in binding_list:
        pscene.rebind(bd, list2dict(lastQ, pscene.gscene.joint_names))
        
new_state = State(*(pscene.get_object_state()+(lastQ, pscene)))
gscene.show_motion(traj, period=0.05)

idx = 1
from_state = new_state.copy(pscene)
to_state = from_state.copy(pscene)
to_state.binding_state = (('brush', 'face', 'goal_bd', 'goal'),)
redundancy_dict = snode_schedule[idx+1].redundancy_dict
redundancy_dict = {obj:{'w': np.pi/2}}
traj, lastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, 
                                                    redundancy_dict=redundancy_dict)
print(success)
gscene.show_motion(traj, period=0.05)

NameError: name 'obj' is not defined

In [23]:
self = mplan
if len(binding_list)!=1:
    raise(RuntimeError("Only single manipulator operation is implemented with moveit!"))
self.update_gscene()

obj_name, ap_name, binder_name, binder_geometry_name = binding_list[0]
redundancy = redundancy_dict[obj_name] if redundancy_dict else None

binder = self.pscene.actor_dict[binder_name]
obj = self.pscene.subject_dict[obj_name]
handle = obj.action_points_dict[ap_name]
point_add, rpy_add = calc_redundancy(redundancy, binder)
T_handle = handle.Toff_lh
T_binder = np.matmul(binder.Toff_lh, SE3(Rot_rpy(rpy_add), point_add))

if len(self.planner.group_names)==1:
    group_name_handle = self.planner.group_names if handle.geometry.link_name in self.urdf_content.parent_map else []
    group_name_binder = self.planner.group_names if binder.geometry.link_name in self.urdf_content.parent_map else []
else:
    group_name_handle = [gname for gname in self.planner.group_names if gname in handle.geometry.link_name]
    group_name_binder = [gname for gname in self.planner.group_names if gname in binder.geometry.link_name]

dual = False
if group_name_binder and not group_name_handle:
    group_name = group_name_binder[0]
    tool, T_tool = binder, T_binder
    target, T_tar = handle, T_handle
elif group_name_handle and not group_name_binder:
    group_name = group_name_handle[0]
    tool, T_tool = handle, T_handle
    target, T_tar = binder, T_binder
else:
    if not self.enable_dual:
        raise(RuntimeError("dual arm motion is not enabled"))
    dual = True
    group_name = "{}_{}".format(group_name_binder[0], group_name_handle[0])
    self.update_gscene(group_name)
    tool, T_tool = handle, T_handle
    target, T_tar = binder, T_binder

T_tar_tool = np.matmul(T_tar, SE3_inv(T_tool))
goal_pose = tuple(T_tar_tool[:3,3]) \
            +tuple(Rotation.from_dcm(T_tar_tool[:3,:3]).as_quat())

if dual:
    dual_planner = self.dual_planner_dict[group_name]
    planner = dual_planner.planner
    from_Q = from_state.Q[dual_planner.idx_pscene_to_mpc]*dual_planner.joint_signs
else:
    planner = self.planner
    if self.need_mapping:
        from_Q = from_state.Q[self.idx_pscene_to_mpc]
    else:
        from_Q = from_state.Q

In [24]:
gscene.show_pose(from_Q)

## Test constrained motion

In [25]:
from pkg.planning.constraint.constraint_common import MotionConstraint

In [26]:
motion_constraint = MotionConstraint([gscene.NAME_DICT["floor"]], True, True, 1e-3)
timeout = 10

In [27]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
mplan.add_constraint(group_name, tool.geometry.link_name, tool.Toff_lh, 
                     motion_constraint=motion_constraint)
trajectory, success = mplan.planner.plan_constrained_py(
    group_name,tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q), timeout=timeout)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (33)


In [28]:
gscene.show_motion(trajectory, period=0.05)

## Disconnect stereo

In [26]:
stereo.disconnnect()